In [1]:
import plotly.express as px #for creating the graphs
from jupyter_dash import JupyterDash #imports Jupyter in our project
#2 dash dependencies are also needed
from dash import dcc 
from dash import html #to render html components in a dash app
from dash.dependencies import Input, Output #this is to be able to perform the callback and actually update the graphs
import pandas as pd
from plotly.subplots import make_subplots
from TimeSeriesClustering import cluster

In [2]:
ts1_df = pd.read_csv('../data/devices1.csv')
ts2_df = pd.read_csv('../data/devices2.csv')
ts3_df = pd.read_csv('../data/devices3.csv')
ts4_df = pd.read_csv('../data/devices4.csv')
data = pd.concat([ts1_df, ts2_df, ts3_df, ts4_df], axis = 0, ignore_index = True)

# data = pd.read_csv('../data/devices1.csv')

In [3]:
list_var = ['battery_voltage', 'current']

In [4]:
#Step 1: create app
app = JupyterDash(__name__)
app._terminate_server_for_port("localhost", 8049)

#Step 2: create layout
app.layout = html.Div([
    html.H1('Raw Data clustering'),
    dcc.Graph(id='double-graph'),
    dcc.Dropdown(id = 'id-dropdown',
                 value = '1',
                 options = [{'label': c, 'value': c} for c in data['battery_id'].unique()]),
    dcc.Dropdown(id = 'var-dropdown',
                 value = 'voltage',
                 options = [{'label': a, 'value': a} for a in list_var])
])

#Step 4: callback
#https://stackoverflow.com/questions/69725119/dash-callback-with-2-input
@app.callback(
    Output(component_id = 'double-graph', component_property = 'figure'),
    [Input(component_id = 'id-dropdown', component_property = 'value'), Input(component_id = 'var-dropdown', component_property = 'value')]
)

#Step 3: plotting function
def update_figure(selected_id, selected_var):
    longest_series, cluster_count, labels, km, bat_df = cluster(data)
    fig = make_subplots(rows=cluster_count, cols=1)
    graph_id = 1
    for label in set(labels):
        for i in range(len(data.battery_id.unique())):
            if(labels[i]==label):
                fig.add_scatter(x=longest_series.index, y=bat_df[i], row=graph_id, col=1)
        graph_id += 1

    fig.update_layout(height=600, width=600)
    return fig

In [5]:
app.run_server(debug=True, mode='external', port=8049)

Dash app running on http://127.0.0.1:8049/
